In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import pandas as pd
import ast
from sklearn.model_selection import train_test_split

# Data Preparation Functions

In [2]:
def create_adjacency_matrices(df):
    df['adjacency_matrix'] = None
    # Step 3: Iterate through Rows and Populate the New Column
    for index, row in df.iterrows(): 
        # Convert String to a real list !
        edges = ast.literal_eval(row['edges'])
        # Determine the number of nodes
        num_nodes = max(max(edge) for edge in edges) + 1
        # Create an empty adjacency matrix
        adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
        # Fill the adjacency matrix based on the edges
        for edge in edges:
            adjacency_matrix[edge[0], edge[1]] = 1
            adjacency_matrix[edge[1], edge[0]] = 1  # Assuming the graph is undirected

        # Assign the adjacency matrix to the new column
        df.at[index, 'adjacency_matrix'] = adjacency_matrix
        
    return df

## Apply Data Preparation 

In [3]:
# Reading the File 
df = pd.read_csv('datacvxpy8.csv')

In [4]:
create_adjacency_matrices(df)
df['node_assignment'] = df['node_assignment'].apply(ast.literal_eval)

In [5]:
df.head()

,n,edges,node_assignment,adjacency_matrix
0,8,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[1, 1, 1, 1, -1, -1, -1, -1]","[[0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, ..."
1,8,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[1, 1, 1, 1, -1, -1, -1, -1]","[[0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, ..."
2,8,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[1, 1, 1, 1, -1, -1, -1, -1]","[[0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, ..."
3,8,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[1, 1, 1, 1, -1, -1, -1, -1]","[[0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, ..."
4,8,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[1, 1, 1, 1, -1, -1, -1, -1]","[[0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, ..."


In [7]:
# Step 3: Split Data
X_train, X_temp, y_train, y_temp = train_test_split(df['adjacency_matrix'].values, df['node_assignment'].values, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
# Convert lists to numpy arrays
X_train = np.array(X_train.tolist()).astype('float32')
y_train = np.array(y_train.tolist()).astype('float32')
X_val = np.array(X_val.tolist()).astype('float32')
y_val = np.array(y_val.tolist()).astype('float32')
X_test = np.array(X_test.tolist()).astype('float32')
y_test = np.array(y_test.tolist()).astype('float32')

In [9]:
X_train.shape

(327, 8, 8)

# Build the GNN model 

In [10]:
# Define GNN model
class GNN(Model):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = layers.Conv1D(16, 1, activation='relu')  # Convolutional layer
        self.conv1 = layers.Conv1D(32, 3, activation='relu')  # Convolutional layer
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')  # Dense layer
        self.dense2 = layers.Dense(8)  # Output layer with 8 units
        self.built = True  # Set model as built

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [11]:
# Initialize model
model = GNN()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile model
model.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for regression task

# Print model summary
model.build((None, 8, 8))  # Manually build the model with input shape
model.summary()


# Train model
model.fit(X_train, y_train, epochs=200,validation_data=(X_val, y_val))

Model: "gnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           multiple                  800       
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  12352     
                                                                 
 dense_1 (Dense)             multiple                  520       
                                                                 
Total params: 13672 (53.41 KB)
Trainable params: 13672 (53.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
11/11 [==============================] - 1s 20ms/step - loss: 0.8812 - val_loss: 0.6565
Epoch 2/200
11/11 [==============================] - 0s 4ms

11/11 [==============================] - 0s 4ms/step - loss: 0.1406 - val_loss: 0.2801
Epoch 74/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1388 - val_loss: 0.2856
Epoch 75/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1392 - val_loss: 0.2818
Epoch 76/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1370 - val_loss: 0.2864
Epoch 77/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1351 - val_loss: 0.2839
Epoch 78/200
11/11 [==============================] - 0s 4ms/step - loss: 0.1338 - val_loss: 0.2862
Epoch 79/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1346 - val_loss: 0.2802
Epoch 80/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1328 - val_loss: 0.2862
Epoch 81/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1311 - val_loss: 0.2812
Epoch 82/200
11/11 [==============================] - 0s 3ms/step - loss: 0.1314 - val_loss: 0.2836
Epoch 83/200


11/11 [==============================] - 0s 3ms/step - loss: 0.0658 - val_loss: 0.2756
Epoch 155/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0663 - val_loss: 0.2749
Epoch 156/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0670 - val_loss: 0.2703
Epoch 157/200
11/11 [==============================] - 0s 4ms/step - loss: 0.0653 - val_loss: 0.2847
Epoch 158/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0654 - val_loss: 0.2781
Epoch 159/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0638 - val_loss: 0.2789
Epoch 160/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0637 - val_loss: 0.2848
Epoch 161/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0628 - val_loss: 0.2821
Epoch 162/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0620 - val_loss: 0.2757
Epoch 163/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0616 - val_loss: 0.2788
Epoc

In [12]:
# Evaluate model 
loss = model.evaluate(X_train, y_train)
print("Average Loss:", loss)

11/11 [==============================] - 0s 1ms/step - loss: 0.0439
Average Loss: 0.04388255625963211


# Predict on X and y of n equal to 8 

In [13]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Define the threshold
threshold = 0.0  

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, -1)

3/3 [==============================] - 0s 0s/step


# Strict Accuracy

Strict accuracy is a metric that measures the percentage of predictions that match the true values exactly. In other words, it evaluates whether all elements of a prediction match the corresponding elements of the true values. For example, if a prediction array [1, 0, 1, 1] is compared to the true array [1, 0, 1, 1], strict accuracy would be 1 (or 100%), indicating that the prediction is entirely correct. However, if any element of the prediction mismatches the true value, strict accuracy would be 0 (or 0%).

In [14]:
def calculate_strict_accuracy(binary_predictions, y):
    """
    Calculate strict accuracy between binary predictions and true values.

    Args:
        binary_predictions (list of arrays): List of binary predictions.
        y (list of arrays): List of true values.

    Returns:
        float: Strict accuracy.
    """
    count_matches = 0
    count_unmatches = 0

    for i in range(len(binary_predictions)):
        # Convert to numpy arrays if not already
        binary_prediction_arr = np.array(binary_predictions[i])
        y_arr = np.array(y[i])
        
        # Check if the elements are equal
        if np.array_equal(binary_prediction_arr, y_arr):
            count_matches += 1
        else:
            count_unmatches += 1
    # Calculate strict accuracy
    strict_accuracy = count_matches / (count_matches + count_unmatches) if (count_matches + count_unmatches) > 0 else 0
    
    return strict_accuracy


In [15]:
strict_accuracy=calculate_strict_accuracy(binary_predictions,y_test)

In [16]:
strict_accuracy

0.7887323943661971

# Average Accuracy

 Average accuracy, on the other hand, calculates the accuracy for eachindividual prediction separately and then averages these accuracies.This approach provides a more nuanced evaluation of the model's performance,as it considers the accuracy of each prediction independently. For instance,if a model makes three predictions [1, 0, 1], [0, 1, 0], and [1, 1, 1],and the true values are [1, 0, 0], [0, 1, 1], and [1, 1, 1], respectively,the average accuracy would be calculated by averaging the accuracies of these individual predictions. This allows for a more granular assessmentof the model's performance across multiple predictions.

In [17]:
def calculate_average_accuracy(binary_predictions, y):
    """
    Calculate accuracy between binary predictions and true values.

    Args:
        binary_predictions (list of arrays): List of binary predictions.
        y (list of arrays): List of true values.

    Returns:
        float: Average accuracy.
    """
    accuracies = []

    for i in range(len(binary_predictions)):
        # Convert to numpy arrays if not already
        binary_prediction_arr = np.array(binary_predictions[i])
        y_arr = np.array(y[i])
        
        # Check if the elements are equal
        accuracy = np.mean(binary_prediction_arr == y_arr)
        accuracies.append(accuracy)
        
    # Calculate average accuracy
    average_accuracy = np.mean(accuracies)
    
    return average_accuracy

In [18]:
average_accuracy=calculate_average_accuracy(binary_predictions,y_test)

In [19]:
average_accuracy

0.9630281690140845